<a href="https://colab.research.google.com/github/hmcgoran/GP16-N2O-Endmember-Model/blob/main/Final_GP16_multiple_regression_Monte_Carlo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final GP16 Multiple Regression Overview (w/ Monte Carlo)
This code uses an isotope mixing model adapted from Peters et al. 2018 which analyzed NO3- end members in the transect. The objective is to quantify the relative contributions of four N2O end members (background, AOA, incomplete denitrification (ID), and Oxygen Deficient Zone (ODZ)) in the GP16 transect. Broadly, the methodology involves defining the delta values of background N2O, AOA-produced N2O, ID-produced N2O, N2O produced in the ODZ, and a multiple linear least squares regression to estimate the relative contributions of each end member.

The Colab is integrated with two Google Sheets: V4_COLAB Final GP16 Regression Model and V4_COLAB Water Mass Delta Definitions. If you want to run this code and make edits to the Colab, you should make a copy of the Colab + two sheets I linked above, and then update the links to the sheet in the Colab. The links needing change are distinguished in a section below using pound signs as follows:
############################################################

worksheet_name = gc.open_by_url('Google Sheet URL')

worksheet_name = gc.open_by_url('Google Sheet URL')

############################################################

The Google Sheet URL can be obtained by clicking share > copy link in the Google Sheet.

The code is based on data from GP16, meaning that all of the sheets referenced are also specific to GP16. To adapt this code to other cases, you would need to link different Google Sheets, change variable names, column names, and anything else that is specific to the region.

# Logistics, Package Imports, Function Definitions, and Google Sheet Integration

First, you will need to give Google Colab permission to access your Drive, which allows the formation of a live connection between Google Sheets and Colab.

In [ ]:
######## Makes it possible to have a live connection between Google Sheets and Colab #######
from google.colab import auth
import gspread
from google.auth import default

# Authenticating to google
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

The next two lines make it possible to import and export to the two Google Sheets mentioned in the overview.

**IF YOU WANT TO RUN THIS COLAB ON YOUR OWN AND MAKE EDITS, CHANGE THESE TWO LINKS AS DESCRIBED IN THE OVERVIEW**

In [ ]:
######################################################################
# Importing data live from the Google Sheets titled: "V4_COLAB Monte Carlo Final GP16 Regression Model"
final_gp16_regression_model = gc.open_by_url(
    'https://docs.google.com/spreadsheets/d/1wGER67HAfrxMDrzXcIlw4ojUQhGLtaZE8gq69vRj5hk/edit?usp=sharing')

# Importing data live from the Google Sheets titled: "V4_COLAB Monte Carlo Water Mass Delta Definitions"
water_mass_delta_definitions = gc.open_by_url(
    'https://docs.google.com/spreadsheets/d/11q_4xOLxm7KRKdBoHDtBVJTJZviCswpKJ7uhs99acH4/edit?usp=sharing')
######################################################################

Next, the functions below need to be imported, some which are used throughout the Colab, and others that are only used once (i.e. csaps and cvxpy). This may take a minute or two, as some of the packages (gsw and csaps) need to be installed.

In [ ]:
import pandas as pd
import numpy as np
!pip install gsw # For the potential density calculations
import gsw as gsw
!pip install csaps # For the cubic spline interpolation
from csaps import csaps
import matplotlib.pyplot as plt # For plotting
%matplotlib inline

The next section defines three functions that are frequently used throughout the colab. See the function descriptions in the comments.

In [ ]:
def import_sheet_as_df(worksheet_name, tab_name):
  '''
  Function that allows you to import sheets and values from
  various Google Sheets tabs. tab_name should be entered as a string and
  worksheet_name = gc.open_by_url('Google Sheet URL'), which should be defined
  prior to calling this function.
  '''
  sheet = worksheet_name.worksheet(tab_name) # Obtains the tab/worksheet of interest from the main Google Sheet that is linked
  rows = sheet.get_all_values() # get_all_values gives a list of rows
  df = pd.DataFrame(rows[1:], columns=rows[0]) # Deletes the row numbers column and makes the first row the headers
  return df

def export_df_to_sheet(worksheet_name, new_tab_name, df_to_export):
  '''
  This function allows you to export a dataframe as a new tab in the Google
  Sheet.  worksheet_name = gc.open_by_url('Google Sheet URL'),
  new_tab_name is a string for what you want to call the new tab, df_to_export
  is the dataframe you are exporting to the sheet
  '''
  new_sheet = worksheet_name.add_worksheet(new_tab_name, rows=400, cols = 85)
  new_sheet.update([df_to_export.columns.values.tolist()] + df_to_export.values.tolist())

def convert_column(dataframe, column_name):
  '''
  This function converts a column in a dataframe to a 1-D array.
  There might be an easier way to do this, but I am not aware of it.
  '''
  prelim_list = list(dataframe[column_name])
  variable_array = [float(i) for i in prelim_list]
  return variable_array

# Preliminary Data Processing and Merges

The following code then merges the N2O dataset, metadata, and water mass percentages into one big dataset by Sample ID. This is done through two merges. Potential density is calculated using the [gsw package](https://teos-10.github.io/GSW-Python/conversions.html) and added as a column to the dataframe before merging with the water mass data.

In [ ]:
# Importing different tabs of the "Final GP16 Regression Model" Google Sheets as dataframes
n2o = import_sheet_as_df(final_gp16_regression_model, "GP16 N2O Data")
water_mass = import_sheet_as_df(final_gp16_regression_model, "Water Mass Data")

# Merge N2O data with water mass data
post_merge = pd.merge(n2o, water_mass, on='Sample ID')

# The next line will export the merged data to Google Sheets under the tab "Post-Merge Data Colab"
# You should comment the following line out if you don't need to export the data (i.e. if it is already there from a prior run)
# export_df_to_sheet(final_gp16_regression_model, 'Post-Merge Data Colab', post_merge)

# Background N2O Definition
Background N2O is the final term we use to describe preformed N2O, which is why "pre" is still used in this code and in the sheet.

## Background Definition - Import Intermediate + Deep Water Mass Definitions

This section extracts all of the intermediate and deep water mass definitions from the Water Mass Delta Definitions Google Sheet and assigns them to variables for later use. If you change a value in the sheet, it will automatically update in the Google Colab.

In [ ]:
inter_deep_def = import_sheet_as_df(water_mass_delta_definitions, "Intermediate/Deep Definitions")

# NOTE: If you change a value in the sheet, it will automatically update when imported to Google Colab
AAIW_N2O = inter_deep_def.loc[0]['[N2O]']
EQPIW_N2O = inter_deep_def.loc[1]['[N2O]']
UCDW_N2O = inter_deep_def.loc[2]['[N2O]']
PDW_N2O = inter_deep_def.loc[3]['[N2O]']
LCDW_N2O = inter_deep_def.loc[4]['[N2O]']
AABW_N2O = inter_deep_def.loc[5]['[N2O]']

AAIW_D15NA = inter_deep_def.loc[0]['d15Na']
EQPIW_D15NA = inter_deep_def.loc[1]['d15Na']
UCDW_D15NA = inter_deep_def.loc[2]['d15Na']
PDW_D15NA = inter_deep_def.loc[3]['d15Na']
LCDW_D15NA = inter_deep_def.loc[4]['d15Na']
AABW_D15NA = inter_deep_def.loc[5]['d15Na']

AAIW_D15NB = inter_deep_def.loc[0]['d15Nb']
EQPIW_D15NB = inter_deep_def.loc[1]['d15Nb']
UCDW_D15NB = inter_deep_def.loc[2]['d15Nb']
PDW_D15NB = inter_deep_def.loc[3]['d15Nb']
LCDW_D15NB = inter_deep_def.loc[4]['d15Nb']
AABW_D15NB = inter_deep_def.loc[5]['d15Nb']

AAIW_D18O = inter_deep_def.loc[0]['dO18']
EQPIW_D18O = inter_deep_def.loc[1]['dO18']
UCDW_D18O = inter_deep_def.loc[2]['dO18']
PDW_D18O = inter_deep_def.loc[3]['dO18']
LCDW_D18O = inter_deep_def.loc[4]['dO18']
AABW_D18O = inter_deep_def.loc[5]['dO18']

AAIW_O2 = inter_deep_def.loc[0]['O2']
EQPIW_O2 = inter_deep_def.loc[1]['O2']
UCDW_O2 = inter_deep_def.loc[2]['O2']
PDW_O2 = inter_deep_def.loc[3]['O2']
LCDW_O2 = inter_deep_def.loc[4]['O2']
AABW_O2 = inter_deep_def.loc[5]['O2']


## Background Definition - Import Thermocline Water Mass Definitions and Interpolate

In order to calculate the preformed delta values for each sample, we have to get a density-specific delta definition for each sample in the thermocline. This code imports the raw data that defines the three thermocline water masses (ESSW, ESPIW, and SPCW) from the Water Mass Delta Definitions Google Sheet. Then, the code interpolates the data using a cubic spline interpolation function (csaps package) between potential density 26-27 in 0.01 increments. If this is confusing, see the next section, which visualizes this code in plots.

In [ ]:
def univariate(dataframe, variable):
  '''
  This function takes the raw data for each N2O variable and
  interpolates it over the 26-27 sigma theta range in 0.01 increments.
  Eventually, these interpolated values are used to help calculate
  preformed values. See https://csaps.readthedocs.io/en/latest/ for
  documentation on the cubic spline approximation function. Currently,
  a smoothing factor of 0.99 is used, however this number can be changed
  to be more or less sensitive to changes in the raw data.
  '''
  x = np.linspace(26, 27, 100)  # Create 100 density values for interpolation
  spline = csaps(dataframe["Density"], dataframe[variable], x, smooth=0.99)
  return spline

# Imports the raw data for each of the three thermocline water masses
essw = import_sheet_as_df(water_mass_delta_definitions, "ESSW Definition")
espiw = import_sheet_as_df(water_mass_delta_definitions, "ESPIW Definition")
spcw = import_sheet_as_df(water_mass_delta_definitions, "SPCW Definition")

density = np.linspace(26, 27, 100) # Defines an array of "x-values" over which we want to interpolate
espiw_df = pd.DataFrame(density, columns=['Density'])  # Create 100 row empty dataframe for interpolated values
essw_df = pd.DataFrame(density, columns=['Density'])
spcw_df = pd.DataFrame(density, columns=['Density'])

variables = ['[N2O]', 'd15Na', 'd15Nb', 'd18O', 'd15Nbulk', 'SP', 'O2'] # Variables to interpolate
for variable in variables:
  espiw_df[variable] = univariate(espiw, variable) # Adds 100-point interpolation to the new dataframe for every variable
  essw_df[variable] = univariate(essw, variable)
  spcw_df[variable] = univariate(spcw, variable)

espiw_df['Density'] = espiw_df['Density'].round(2) # Rounds the values to two decimal places
essw_df['Density'] = essw_df['Density'].round(2)
spcw_df['Density'] = spcw_df['Density'].round(2)

print(espiw_df)
print(essw_df)
print(spcw_df)

# export_df_to_sheet(water_mass_delta_definitions, 'ESSW Definitions Post-Interpolation (12/17)', essw_df)
# export_df_to_sheet(water_mass_delta_definitions, 'ESPIW Definitions Post-Interpolation (12/17)', espiw_df)
# export_df_to_sheet(water_mass_delta_definitions, 'SPCW Definitions Post-Interpolation (12/17)', spcw_df)

## Background Definition - Thermocline Interpolation and Profile Plots

This section visualizes what was done in the last section, where each thermocline water mass variable was interpolated from 26-27 sigma theta.

In [ ]:
def thermocline_plots(row, col, min, max, watermass_df, col_name, spline_df, var_label):
  '''
  This is a general function for plotting the interpolations and raw data values
  for each water mass and each variable. Row = plot row, col = plot col,
  min = x-axis minimum, max = x-axis maximum, watermass_df = imported dataframe
  containing the raw values, col_name = the variable of interest, spline_df =
  the dataframe that holds the interpolated value, var_label = name for the x-axis
  '''
  watermass_df[col_name] = watermass_df[col_name].astype(float) # Raw Data Values
  watermass_df['Density'] = watermass_df['Density'].astype(float) # Density corresponding to raw data
  ax[row, col].plot(watermass_df[col_name], watermass_df['Density'], 'bo', markersize=3)  # Plot original points
  ax[row, col].plot(spline_df[col_name], spline_df['Density'], 'x-', c='orange', markersize=3)  # Plot spline
  ax[row, col].set_xlim(min, max)
  ax[row, col].set_ylabel('Potential Density')
  ax[row, col].set_xlabel(var_label)

fig, ax = plt.subplots(3, 5, sharey='row') # Make the y-axis the same for all rows
fig.set_size_inches(14, 14) # Size of the panel
fig.subplots_adjust(hspace = 0.3, wspace = 0.2) # Space between plots

# ESSW Plots
ax[0, 0].invert_yaxis()
thermocline_plots(0, 0, 10, 65, essw, '[N2O]', essw_df, "[N2O] ESSW")
thermocline_plots(0, 1, 40, 65, essw, 'd18O', essw_df, "d18O ESSW")
thermocline_plots(0, 2, 7.5, 20, essw, 'd15Na', essw_df, "d15Na ESSW")
thermocline_plots(0, 3, -4, 3, essw, 'd15Nb', essw_df, "d15Nb ESSW")
thermocline_plots(0, 4, 0, 210, essw, 'O2', essw_df, "O2 ESSW")

# ESPIW Plots
ax[1, 0].invert_yaxis()
thermocline_plots(1, 0, 10, 65, espiw, '[N2O]', espiw_df, "[N2O] ESPIW")
thermocline_plots(1, 1, 40, 65, espiw, 'd18O', espiw_df, "d18O ESPIW")
thermocline_plots(1, 2, 7.5, 20, espiw, 'd15Na', espiw_df, "d15Na ESPIW")
thermocline_plots(1, 3, -4, 3, espiw, 'd15Nb', espiw_df, "d15Nb ESPIW")
thermocline_plots(1, 4, 0, 210, espiw, 'O2', espiw_df, "O2 ESPIW")

# SPCW Plots
ax[2, 0].invert_yaxis()
thermocline_plots(2, 0, 10, 65, spcw, '[N2O]', spcw_df, "[N2O] SPCW")
thermocline_plots(2, 1, 40, 65, spcw, 'd18O', spcw_df, "d18O SPCW")
thermocline_plots(2, 2, 7.5, 20, spcw, 'd15Na', spcw_df, "d15Na SPCW")
thermocline_plots(2, 3, -4, 3, spcw, 'd15Nb', spcw_df, "d15Nb SPCW")
thermocline_plots(2, 4, 0, 210, spcw, 'O2', spcw_df, "O2 SPCW")

plt.show()

## Background Definition - Calculate Sample-Specific Preformed Values

This section takes all of the previously defined data to calculate sample-by-sample preformed d18O, d15Na, d15Nb, and O2. First, the water mass percentage values from Peters 2018 are imported as lists.

In [ ]:
# post_merge = import_sheet_as_df(final_gp16_regression_model, "Post-Merge Data Colab")

def water_mass_percent(column):
  '''
  This function takes the column of water percent values for the specified
  water mass from the post-merge dataset and divides all values by 100
  so that the percentages are converted to proportions and able to be
  incorporated into the preformed calculation.
  '''
  pct_import = post_merge[column].tolist()
  pct_list = [float(num) / 100 for num in pct_import] # Convert the proportions from Peters et al. 2018 to percentages
  return pct_list

essw_pct = water_mass_percent('ESSW_PCT')
espiw_pct = water_mass_percent('ESPIW_PCT')
spcw_pct = water_mass_percent('SPCW_PCT')
aaiw_pct = water_mass_percent('AAIW_PCT')
eqpiw_pct = water_mass_percent('EQPIW_PCT')
ucdw_pct = water_mass_percent('UCDW_PCT')
pdw_pct = water_mass_percent('PDW_PCT')
lcdw_pct = water_mass_percent('LCDW_PCT')
aabw_pct = water_mass_percent('AABW_PCT')

Next, isotope mass balance is used to calculate the preformed values for each sample using this equation: Σ over all water masses (water mass % * water mass definition * [N2O]) / Σ over all water masses (water mass % * [N2O])

In [ ]:
d18O_pre = [] # Set up empty lists to hold the calculated preformed values
d15Na_pre = []
d15Nb_pre = []
o2_pre = []
samp_density = post_merge['Potdens'].astype(float).round(2).tolist()

for i in range(len(samp_density)): # Loops through every sample in the entire dataset
  if 26 <= samp_density[i] <= 27: # If the sample is in the thermocline
    row_espiw = espiw_df.loc[espiw_df['Density'] == samp_density[i]]  # Obtains the row from the espiw_df for the respective density
    row_essw = essw_df.loc[essw_df['Density'] == samp_density[i]]
    row_spcw = spcw_df.loc[spcw_df['Density'] == samp_density[i]]
  else:
    # If the sample density is not in the thermocline (26-27 sigma theta), then we make the values 0
    row_espiw = pd.DataFrame(np.zeros((1,6)), columns = ['Density', '[N2O]', 'd15Na','d15Nb', 'd18O', 'O2'])
    row_essw = pd.DataFrame(np.zeros((1,6)), columns = ['Density', '[N2O]', 'd15Na','d15Nb', 'd18O', 'O2'])
    row_spcw = pd.DataFrame(np.zeros((1,6)), columns = ['Density', '[N2O]', 'd15Na','d15Nb', 'd18O', 'O2'])

  # Calculate the denominator, which is just water mass percent * [N2O] all summed together
  denominator = (
    float(essw_pct[i]) * float(row_essw['[N2O]']) +
    float(espiw_pct[i]) * float(row_espiw['[N2O]']) +
    float(spcw_pct[i]) * float(row_spcw['[N2O]']) +
    float(aaiw_pct[i]) * float(AAIW_N2O) +
    float(eqpiw_pct[i]) * float(EQPIW_N2O) +
    float(ucdw_pct[i]) * float(UCDW_N2O) +
    float(pdw_pct[i]) * float(PDW_N2O) +
    float(lcdw_pct[i]) * float(LCDW_N2O) +
    float(aabw_pct[i]) * float(AABW_N2O)
    )

  # d18O preformed calculation for one sample
  d18O_pre_calc = (
    float(essw_pct[i]) * float(row_essw['[N2O]']) * float(row_essw['d18O']) +
    float(espiw_pct[i]) * float(row_espiw['[N2O]']) * float(row_espiw['d18O']) +
    float(spcw_pct[i]) * float(row_spcw['[N2O]']) * float(row_spcw['d18O']) +
    float(aaiw_pct[i]) * float(AAIW_N2O) * float(AAIW_D18O) +
    float(eqpiw_pct[i]) * float(EQPIW_N2O) * float(EQPIW_D18O) +
    float(ucdw_pct[i]) * float(UCDW_N2O) * float(UCDW_D18O) +
    float(pdw_pct[i]) * float(PDW_N2O) * float(PDW_D18O) +
    float(lcdw_pct[i]) * float(LCDW_N2O) * float(LCDW_D18O) +
    float(aabw_pct[i]) * float(AABW_N2O) * float(AABW_D18O)
    ) / denominator
  d18O_pre_calc = round(d18O_pre_calc, 2)
  d18O_pre.append(d18O_pre_calc)

  # d15Na preformed calculation for one sample
  d15Na_pre_calc = (
    float(essw_pct[i]) * float(row_essw['[N2O]']) * float(row_essw['d15Na']) +
    float(espiw_pct[i]) * float(row_espiw['[N2O]']) * float(row_espiw['d15Na']) +
    float(spcw_pct[i]) * float(row_spcw['[N2O]']) * float(row_spcw['d15Na']) +
    float(aaiw_pct[i]) * float(AAIW_N2O) * float(AAIW_D15NA) +
    float(eqpiw_pct[i]) * float(EQPIW_N2O) * float(EQPIW_D15NA) +
    float(ucdw_pct[i]) * float(UCDW_N2O) * float(UCDW_D15NA) +
    float(pdw_pct[i]) * float(PDW_N2O) * float(PDW_D15NA) +
    float(lcdw_pct[i]) * float(LCDW_N2O) * float(LCDW_D15NA) +
    float(aabw_pct[i]) * float(AABW_N2O) * float(AABW_D15NA)
    ) / denominator
  d15Na_pre_calc = round(d15Na_pre_calc, 2)
  d15Na_pre.append(d15Na_pre_calc)

  # d15Nb preformed calculation for one sample
  d15Nb_pre_calc = (
    float(essw_pct[i]) * float(row_essw['[N2O]']) * float(row_essw['d15Nb']) +
    float(espiw_pct[i]) * float(row_espiw['[N2O]']) * float(row_espiw['d15Nb']) +
    float(spcw_pct[i]) * float(row_spcw['[N2O]']) * float(row_spcw['d15Nb']) +
    float(aaiw_pct[i]) * float(AAIW_N2O) * float(AAIW_D15NB) +
    float(eqpiw_pct[i]) * float(EQPIW_N2O) * float(EQPIW_D15NB) +
    float(ucdw_pct[i]) * float(UCDW_N2O) * float(UCDW_D15NB) +
    float(pdw_pct[i]) * float(PDW_N2O) * float(PDW_D15NB) +
    float(lcdw_pct[i]) * float(LCDW_N2O) * float(LCDW_D15NB) +
    float(aabw_pct[i]) * float(AABW_N2O) * float(AABW_D15NB)
    ) / denominator
  d15Nb_pre_calc = round(d15Nb_pre_calc, 2)
  d15Nb_pre.append(d15Nb_pre_calc)

  # O2 preformed calculation for one sample
  o2_pre_calc = (
    float(essw_pct[i]) * float(row_essw['O2']) +
    float(espiw_pct[i]) * float(row_espiw['O2']) +
    float(spcw_pct[i]) * float(row_spcw['O2']) +
    float(aaiw_pct[i]) * float(AAIW_O2) +
    float(eqpiw_pct[i]) * float(EQPIW_O2) +
    float(ucdw_pct[i]) * float(UCDW_O2) +
    float(pdw_pct[i]) * float(PDW_O2)  +
    float(lcdw_pct[i]) * float(LCDW_O2)  +
    float(aabw_pct[i]) * float(AABW_O2)
    )
  o2_pre_calc = round(o2_pre_calc, 2)
  o2_pre.append(o2_pre_calc)

print(f"Mean Preformed d18O: {round(sum(d18O_pre)/len(d18O_pre), 2)}", "\n"f"Min/Max Preformed d18O: {min(d18O_pre)} to {max(d18O_pre)}")
print(f"Preformed d18O Values: {d18O_pre}")

print("\n" f"Mean Preformed d15Na: {round(sum(d15Na_pre)/len(d15Na_pre), 2)}", "\n"f"Min/Max Preformed d15Na: {min(d15Na_pre)} to {max(d15Na_pre)}")
print(f"Preformed d15Na Values: {d15Na_pre}")

print("\n" f"Mean Preformed d15Nb: {round(sum(d15Nb_pre)/len(d15Nb_pre), 2)}", "\n"f"Min/Max Preformed d15Nb: {min(d15Nb_pre)} to {max(d15Nb_pre)}")
print(f"Preformed d15Nb Values: {d15Nb_pre}")
print("\n" f"Preformed O2 Values: {o2_pre}")

# Model Run

To conduct the mathematical optimization, we must import the observed N2O data, and then subsequently define the AOA and ODZ N2O delta values as constants. See the paper for more detail on the AOA and ODZ definitions.

In [ ]:
# Import observed data values
n2o_obs =  post_merge['n2o_nm'].tolist()
d18O_obs = post_merge['d18o'].tolist()
d15Na_obs = post_merge['alpha'].tolist()
d15Nb_obs = post_merge['beta'].tolist()

########## AOA End Member Definition #############
# From Santoro et al. 2021, corrected for substrate
# D18O_AOA = 34.0 # Value as of April 2025
# D18O_AOA = 46.0 # d18O value for sensitivity test
# D18O_AOA =54.0 # d18O value for sensitivity test
D15NA_AOA = 30.3
D15NB_AOA = 0

########## ID End Member Definition #############
# Determine the ID definition based on GP16 NO3- data (point-by-point) and applied isotope effects
d18o_no3 = post_merge['d18O-NO3 AVG (FINAL)'].tolist()
d15n_no3 = post_merge['d15N-NO3 AVG (FINAL)'].tolist()

print(f"D18O NO3- : {d18o_no3}")
print(f"D15Nbulk NO3- : {d15n_no3}")

d18o_id = np.array(d18o_no3, dtype=float) + 36
d15na_id = np.array(d15n_no3, dtype=float) - 0
d15nb_id = np.array(d15n_no3, dtype=float) - 22
# d15na_id = np.array(d15n_no3, dtype=float) - 10 # d15na_id for sensitivity test
# d15nb_id = np.array(d15n_no3, dtype=float) - 10 # d15nb_id for sensitivity test
print("\n" f"D18O ID Definition: {d18o_id}")
print(f"D15NA ID Definition: {d15na_id}")
print(f"D15NB ID Definition: {d15nb_id}")

########## CD End Member Definition #####################
# Determine the CD definition based on collected data
odz_n2o = post_merge[(post_merge['OXYGEN'].astype(float) < 5) & (post_merge['Potdens'].astype(float) > 26.2) & (post_merge['Potdens'].astype(float) < 26.8)]
D18O_CD = (sum(odz_n2o['d18o'].astype(float) * odz_n2o['n2o_nm'].astype(float))) / sum(odz_n2o['n2o_nm'].astype(float))
D18O_SD_CD = np.sqrt((sum((odz_n2o['d18o'].astype(float)-D18O_CD)**2 * odz_n2o['n2o_nm'].astype(float)))
                      / ((len(odz_n2o['n2o_nm'])-1) * sum(odz_n2o['n2o_nm'].astype(float)) / len(odz_n2o['n2o_nm'])))

D15NA_CD = (sum(odz_n2o['alpha'].astype(float) * odz_n2o['n2o_nm'].astype(float))) / sum(odz_n2o['n2o_nm'].astype(float))
D15NA_SD_CD = np.sqrt((sum((odz_n2o['alpha'].astype(float)-D15NA_CD)**2 * odz_n2o['n2o_nm'].astype(float)))
                      / ((len(odz_n2o['n2o_nm'])-1) * sum(odz_n2o['n2o_nm'].astype(float)) / len(odz_n2o['n2o_nm'])))

D15NB_CD = (sum(odz_n2o['beta'].astype(float) * odz_n2o['n2o_nm'].astype(float))) / sum(odz_n2o['n2o_nm'].astype(float))
D15NB_SD_CD = np.sqrt((sum((odz_n2o['beta'].astype(float)-D15NB_CD)**2 * odz_n2o['n2o_nm'].astype(float)))
                      / ((len(odz_n2o['n2o_nm'])-1) * sum(odz_n2o['n2o_nm'].astype(float)) / len(odz_n2o['n2o_nm'])))

print("\n" f"D18O CD Definition: {str(round(D18O_CD, 2))}, std({str(round(D18O_SD_CD, 2))})")
print(f"D15NA CD Definition: {str(round(D15NA_CD, 2))}, std({str(round(D15NA_SD_CD, 2))})")
print(f"D15NB CD Definition: {str(round(D15NB_CD, 2))}, std({str(round(D15NB_SD_CD, 2))})")


At this point, we now have all the necessary data to apply the multiple linear least squares regression function. The operations carried out below are from the CVXPY package documented here: https://www.cvxpy.org/. The least squares function allows you to constrain the variables to be positive and sum to 1, as shown in the code below. Setting this constraint (along with the constraint that the coefficients have to be positive) ensures that we can appropriately account for all of the N2O concentration at each point. Using a least squares function that does not constrain the coefficients to sum to one leads to high residual concentrations and unintuitive results (more on this in the write-up).

In [ ]:
# CVXPY 1.4
!pip install cvxpy==1.6
import cvxpy as cp
!pip show cvxpy

In [ ]:
# Create empty dataframe to store results
ratios = pd.DataFrame(columns = ['Preformed Ratio', 'AOA Ratio', 'ID Ratio', 'CD Ratio'])

for i in range(len(n2o_obs)):
  A = np.array([[d15Na_pre[i], D15NA_AOA, d15na_id[i], D15NA_CD], [d15Nb_pre[i], D15NB_AOA, d15nb_id[i], D15NB_CD], [d18O_pre[i], D18O_AOA, d18o_id[i], D18O_CD]])
  b = np.array([d15Na_obs[i], d15Nb_obs[i], d18O_obs[i]])

  x = cp.Variable(4, nonneg=True) # Number of coefficients that are being guessed, also equal to number of rows in A
  objective = cp.Minimize(cp.sum_squares(A @ x-b)) # Minimizes the sum of the squares to optimize the coefficient guess
  constraints = [cp.sum(x) == 1] # Constrains the coefficient predictions to be >= 0 and sum to 1
  prob  = cp.Problem(objective, constraints)
  result = prob.solve()

  ratios_to_df = pd.DataFrame([x.value], columns = ratios.columns)
  ratios = pd.concat([ratios, ratios_to_df], ignore_index = True) # Appends new sample onto dataframe

print(ratios) # The ratios variable holds the result

# Monte Carlo

## Monte Carlo that bypasses cases when CVXPY fails

In [ ]:
# Monte Carlo to Catch Exceptions
def montecarlo_w_except(n, post_merge,
                              D18O_AOA, D15NA_AOA, D15NB_AOA,
                              d18o_id, d15na_id, d15nb_id,
                              D18O_CD, D18O_CD_SD, D15NA_CD, D15NA_CD_SD, D15NB_CD, D15NB_CD_SD):
      """
      Function montecarlo_w_except inputs:
      n = number of Monte Carlo iterations
      post_merge = growing dataframe of observed and other compiled data
      D18O_AOA = D18O AOA Definition
      D15NA_AOA = D15NA AOA Definition
      D15NB_AOA = D15NB AOA Definition

      d18o_id = D18O ID Definition
      d15na_id = D15NA ID Definition
      d15nb_id = D15NB ID Definition

      D18O_CD = D18O CD Definition
      D18O_CD_SD = D18O CD Definition Standard Deviation
      D15NA_CD = D15NA CD Definition
      D15NA_CD_SD = D15NA CD Definition Standard Deviation
      D15NB_CD = D15NB CD Definition
      D15NB_CD_SD = D15NB CD Definition Standard Deviation

      Function runs a Monte Carlo simulation varying the ODZ end member by generating random parameter values from a normal
      distribution defined by the respective standard deviations. Accounts for cases in which
      the solver is unable to obtain a solution for the given parameters. Returns the average
      standard deviation for each of the four end members as well as the average standard deviation
      for each sample. Originally, we were going to vary all four end members; however, to avoid propogating
      more error than necessary we eventually limited the variation to the complete denitrification/ODZ/partial
      consumption) end members.
      """
      # Initialize four empty arrays that will hold the end member ratio results for each Monte Carlo iteration
      pre_array = np.empty((len(post_merge["n2o_nm"]), 0))
      aoa_array = np.empty((len(post_merge["n2o_nm"]), 0))
      id_array = np.empty((len(post_merge["n2o_nm"]), 0))
      cd_array = np.empty((len(post_merge["n2o_nm"]), 0))
      rand_parameters = np.empty((3, 0))

      iterations = 0 # Count how many successful iterations there are within n iterations

      for i in range(n): # loop through n Monte Carlo iterations
          # Generate perturbed CD definitions for this particular MC iteration
          d18o_cd_perturb = np.random.normal(D18O_CD, D18O_CD_SD)
          d15na_cd_perturb = np.random.normal(D15NA_CD, D15NA_CD_SD)
          d15nb_cd_perturb = np.random.normal(D15NB_CD, D15NB_CD_SD)

          # To store the results from this specific iteration
          parameters_i = [d18o_cd_perturb, d15na_cd_perturb, d15nb_cd_perturb]
          pre_array_column_i = []
          aoa_array_column_i = []
          id_array_column_i = []
          cd_array_column_i = []

          crashed = False # Set crashed = False to use as a conditional statement later on
          for samp_num in range(len(post_merge["n2o_nm"])): # Looping through all the collected samples for one Monte Carlo iteration
                try: # Attempt to solve the optimization problem
                    A = np.array([[d15Na_pre[samp_num], D15NA_AOA, d15na_id[samp_num], d15na_cd_perturb],
                                  [d15Nb_pre[samp_num], D15NB_AOA, d15nb_id[samp_num], d15nb_cd_perturb],
                                  [d18O_pre[samp_num], D18O_AOA, d18o_id[samp_num], d18o_cd_perturb]])
                    b = np.array([d15Na_obs[samp_num], d15Nb_obs[samp_num], d18O_obs[samp_num]])

                    x = cp.Variable(4, nonneg=True) # Number of coefficients, also equal to number of rows in A
                    objective = cp.Minimize(cp.sum_squares(A @ x-b)) # Minimizes the sum of the squares to optimize the coefficient guess
                    constraints = [cp.sum(x) == 1] # Constrains the coefficient predictions to be >= 0 and sum to 1
                    prob  = cp.Problem(objective, constraints)
                    result = prob.solve()

                    # Add the new result to their respective arrays
                    pre_array_column_i = np.append(pre_array_column_i, x.value[0])
                    aoa_array_column_i = np.append(aoa_array_column_i, x.value[1])
                    id_array_column_i = np.append(id_array_column_i, x.value[2])
                    cd_array_column_i = np.append(cd_array_column_i, x.value[3])

                except cp.SolverError as e: # If there is a Solver Error, print an error message, set crashed = True, terminate current for loop and move on to next Monte Carlo simulation with new parameters
                    print(f"Solver failed during iteration {i}, \n sample {samp_num}, \n d15Na_obs {d15Na_obs[samp_num]}, \n d15Na_pre {d15Na_pre[samp_num]}, \n d15Na_id {d15na_id[samp_num]}, \n d15Na_pc {d15na_cd_perturb}, \n d15Nb_obs {d15Nb_obs[samp_num]}, \n d15Nb_pre {d15Nb_pre[samp_num]}, \n d15Nb_id {d15nb_id[samp_num]}, \n d15Nb_pc {d15nb_cd_perturb}, \n d18O_obs {d18O_obs[samp_num]}, \n d18O_pre {d18O_pre[samp_num]}, \n d18O_id {d18o_id[samp_num]}, \n d18O_pc {d18o_cd_perturb} \n {e}")
                    crashed = True
                    break

          if not crashed and iterations < 1000: # If the given MC iteration was successful, add new column of ratios onto growing array of MC iteration results
            iterations += 1
            rand_parameters = np.column_stack((rand_parameters, parameters_i))
            pre_array = np.column_stack((pre_array, pre_array_column_i))
            aoa_array = np.column_stack((aoa_array, aoa_array_column_i))
            id_array = np.column_stack((id_array, id_array_column_i))
            cd_array = np.column_stack((cd_array, cd_array_column_i))

      # Export MC ratios for each end member to Google Sheets and remember to change the sheet number to export
      export_df_to_sheet(final_gp16_regression_model, 'MC Parameters 10', pd.DataFrame(rand_parameters, columns=["Random Parameters"] * (iterations)))
      export_df_to_sheet(final_gp16_regression_model, 'MC Pre Array 10', pd.DataFrame(pre_array, columns=["Pre Iteration Ratio"] * (iterations)))
      export_df_to_sheet(final_gp16_regression_model, 'MC AOA Array 10', pd.DataFrame(aoa_array, columns=["AOA Iteration Ratio"] * (iterations)))
      export_df_to_sheet(final_gp16_regression_model, 'MC ID Array 10', pd.DataFrame(id_array, columns=["ID Iteration Ratio"] * (iterations)))
      export_df_to_sheet(final_gp16_regression_model, 'MC CD Array 10', pd.DataFrame(cd_array, columns=["CD Iteration Ratio"] * (iterations)))

      pre_stdevs = np.std(pre_array, axis = 1) # Take standard deviations of every row (which corresponds to a standard deviation for each sample)
      aoa_stdevs = np.std(aoa_array, axis = 1)
      id_stdevs = np.std(id_array, axis = 1)
      cd_stdevs = np.std(cd_array, axis = 1)

      pre_array_stdev = np.mean(pre_stdevs) # Find average standard deviation for each end member
      aoa_array_stdev = np.mean(aoa_stdevs)
      id_array_stdev = np.mean(id_stdevs)
      cd_array_stdev = np.mean(cd_stdevs)

      print(f"Successful Iterations: {iterations}")
      print(f"Average Preformed Ratio Standard Deviation: {pre_array_stdev}")
      print(f"Average AOA Ratio Standard Deviation: {aoa_array_stdev}")
      print(f"Average Incomplete Denitrification Ratio Standard Deviation: {id_array_stdev}")
      print(f"Average Partial Consumption Ratio Standard Deviation: {cd_array_stdev}")

      return pre_stdevs, aoa_stdevs, id_stdevs, cd_stdevs, pre_array_stdev, aoa_array_stdev, id_array_stdev, cd_array_stdev

In [ ]:
# Monte Carlo perturbing only ID and CD and d18O AOA at 46
# D18O_AOA = 46.0  # Currently have added 12 to account for 18O of oxygen in deep ocean
D18O_AOA = 34.0  # Currently have added 12 to account for 18O of oxygen in deep ocean
D18O_AOA_SD = 0.9 # Standard deviations of original measurements
D15NA_AOA = 30.3
D15NA_AOA_SD = 1.6
D15NB_AOA = 0
D15NB_AOA_SD = 1.6

D18O_CD = 70.21
D18O_CD_SD = 9.41
D15NA_CD = 23.96
D15NA_CD_SD = 6.72
D15NB_CD = -2.08
D15NB_CD_SD = 3.39

[pre_stdevs, aoa_stdevs, id_stdevs, cd_stdevs, pre_array_stdev, aoa_array_stdev, id_array_stdev, cd_array_stdev] = montecarlo_w_except(1020, post_merge,
                           D18O_AOA, D15NA_AOA, D15NB_AOA,
                           d18o_id, d15na_id, d15nb_id,
                           D18O_CD, D18O_CD_SD, D15NA_CD, D15NA_CD_SD, D15NB_CD, D15NB_CD_SD)


# Validation

Below are two ways of validating the results. The first is a back-calculation of the delta values, which uses the ratios outputted by the mathematical function and the original pre, prod, and ODZ delta definitions to back-calculate a delta value for the sample to see how close the prediction is from the observed value.

In [ ]:
# Back-Calculating the Delta Values
pre_ratios = list(ratios['Preformed Ratio'])
aoa_ratios = list(ratios['AOA Ratio'])
id_ratios = list(ratios['ID Ratio'])
cd_ratios = list(ratios['CD Ratio'])

d18O_backcalc = []
d15Na_backcalc = []
d15Nb_backcalc = []
d18O_errors = []
d15Na_errors = []
d15Nb_errors = []

for i in range(len(pre_ratios)):
  # Back-calculate the three delta values
  d18O_predict = pre_ratios[i] * d18O_pre[i] + aoa_ratios[i] * D18O_AOA + id_ratios[i] * d18o_id[i] + cd_ratios[i] * D18O_CD
  d15Na_predict = pre_ratios[i] * d15Na_pre[i] + aoa_ratios[i] * D15NA_AOA + id_ratios[i] * d15na_id[i] + cd_ratios[i] * D15NA_CD
  d15Nb_predict = pre_ratios[i] * d15Nb_pre[i] + aoa_ratios[i] * D15NB_AOA + id_ratios[i] * d15nb_id[i] + cd_ratios[i] * D15NB_CD

  # Find the difference between the back-calculation and the observed value
  d18O_error = float(d18O_obs[i]) - d18O_predict
  d15Na_error = float(d15Na_obs[i]) - d15Na_predict
  d15Nb_error = float(d15Nb_obs[i]) - d15Nb_predict

  d18O_backcalc.append(d18O_predict) # Append the back-calculation to the back-calculation list
  d15Na_backcalc.append(d15Na_predict)
  d15Nb_backcalc.append(d15Nb_predict)

  d18O_errors.append(d18O_error) # Append the error to the error list
  d15Na_errors.append(d15Na_error)
  d15Nb_errors.append(d15Nb_error)

print(f"d18O Back-Calculation Errors: {d18O_errors}")
print(f"d15Na Back-Calculation Errors: {d15Na_errors}")
print(f"d15Nb Back-Calculation Errors: {d15Nb_errors}")

The second validation approach is to evaluate the relationship between consumed oxygen and the model's estimation of produced N2O. If performing properly, we would expect the relationship to be positive. Consumed oxygen is defined as preformed O2 (which is calculated) - observed O2. The plots are shown below.

In [ ]:
# Consumed O2 vs Produced N2O Plots
consumed_o2 = np.array(o2_pre) - post_merge['OXYGEN'].astype(float)
fig, ax = plt.subplots(1, 2) # Make two plots
fig.set_size_inches(7, 3) # Size of the panel
fig.subplots_adjust(hspace = 0.5, wspace = 0.2) # Space between plots

ax[0].plot(consumed_o2, aoa_ratios, 'bo', markersize=3)  # Plot original points
ax[0].set_ylabel('AOA Produced N2O Ratio')
ax[0].set_xlabel('O2 Consumed (umol/L)')

n2o_obs = [float(num) for num in n2o_obs]
ax[1].plot(consumed_o2, (np.array(aoa_ratios) * n2o_obs), 'bo', markersize=3)  # Plot original points
ax[1].set_ylabel('AOA Produced [N2O]')
ax[1].set_xlabel('O2 Consumed (umol/L)')
plt.show()

# Export Model Results and Validation Results to Google Sheets

Lastly, because new data was generated through all of the above code, this section appends the new data onto the merged dataframe from the start as columns. Finally, this new dataframe is exported as a new tab to the main Google Sheet.

**Remember to change the name of the new tab to be created in the last line of this section, otherwise you will get an error because the tab already exists.**

In [ ]:
# Add all new data to growing dataframe (post_merge), export to Google Sheets at the end
n2o_obs = np.array(n2o_obs, dtype=np.float32)
post_merge['Preformed Conc'] = np.array(pre_ratios) * n2o_obs
post_merge['AOA Conc'] = np.array(aoa_ratios) * n2o_obs
post_merge['ID Conc'] = np.array(id_ratios) * n2o_obs
post_merge['ODZ Conc'] = np.array(cd_ratios) * n2o_obs
post_merge['Preformed d18O'] = d18O_pre
post_merge['Preformed d15Na'] = d15Na_pre
post_merge['Preformed d15Nb'] = d15Nb_pre
post_merge['Preformed O2'] = o2_pre
post_merge['Consumed O2'] = consumed_o2
post_merge['Preformed Ratio'] = pre_ratios
post_merge['AOA Ratio'] = aoa_ratios
post_merge['ID Ratio'] = id_ratios
post_merge['ODZ Ratio'] = cd_ratios
post_merge['Residual N2O'] = n2o_obs - (pre_ratios * n2o_obs + aoa_ratios * n2o_obs + id_ratios * n2o_obs + cd_ratios * n2o_obs)
post_merge['d18O Back Calc'] = d18O_backcalc
post_merge['d15Na Back Calc'] = d15Na_backcalc
post_merge['d15Nb Back Calc'] = d15Nb_backcalc
post_merge['d18O Back Calc Error'] = d18O_errors
post_merge['d15Na Back Calc Error'] = d15Na_errors
post_merge['d15Nb Back Calc Error'] = d15Nb_errors

# post_merge['Preformed Mean Stdev'] = pre_stdevs
# post_merge['AOA Mean Stdev'] = aoa_stdevs
# post_merge['ID Mean Stdev'] = id_stdevs
# post_merge['CD Mean Stdev'] = cd_stdevs

# post_merge['Preformed Ratio + Mean Stdev'] = pre_stdevs + pre_ratios
# post_merge['AOA Ratio + Mean Stdev'] = aoa_stdevs + aoa_ratios
# post_merge['ID Ratio + Mean Stdev'] = id_stdevs + id_ratios
# post_merge['CD Ratio + Mean Stdev'] = cd_stdevs + cd_ratios

# post_merge['Preformed Ratio - Mean Stdev'] = pre_ratios - pre_stdevs
# post_merge['AOA Ratio - Mean Stdev'] = aoa_ratios - aoa_stdevs
# post_merge['ID Ratio - Mean Stdev'] = id_ratios - id_stdevs
# post_merge['CD Ratio - Mean Stdev'] = cd_ratios - cd_stdevs

# Change the sheet name to export
export_df_to_sheet(final_gp16_regression_model, '11 Final Product Colab', post_merge)


# Plotting Results
Plotting results interpolated over density and longitude


In [ ]:
# MAIN FIGURE IN PAPER
# Center is base case, two sides are base case +/- mean state
mc_sheet = import_sheet_as_df(final_gp16_regression_model, "10 Final Product Colab")
odz_n2o = mc_sheet[(mc_sheet['OXYGEN'].astype(float) < 5) & (mc_sheet['Potdens'].astype(float) > 26.2) & (mc_sheet['Potdens'].astype(float) < 26.8)]

from matplotlib import tri
from sklearn.preprocessing import MinMaxScaler
from google.colab import files

# NORMALIZE DATA BEFORE TRIANGULATION
scaler = MinMaxScaler()
scaled_long = scaler.fit_transform(mc_sheet['LONGITUDE_x'].astype(float).values.reshape(-1,1))
scaled_depth = scaler.fit_transform(mc_sheet['Potdens'].astype(float).values.reshape(-1,1))

triang=tri.Triangulation(scaled_long.flatten(), scaled_depth.flatten())
triang_analyze=tri.TriAnalyzer(triang)
mask=triang_analyze.get_flat_tri_mask()
triang.set_mask(mask)

def mc_plot(plot_name, variable, plot_row, plot_column):
  cntr2 = ax[plot_row,plot_column].tricontourf(mc_sheet['LONGITUDE_x'].astype(float),mc_sheet['Potdens'].astype(float),mc_sheet[variable].astype(float), triangles=triang.triangles, mask=triang.mask,levels=np.linspace(-0.35,1.35,51,endpoint=True), extend="neither", cmap="rainbow", vmin=-0.35, vmax=1.35, fontsize=15)
  # colorbar = fig.colorbar(cntr2, ax=ax[plot_row,plot_column])
  # colorbar.ax.tick_params(labelsize=11)

  dens_cntr = ax[plot_row,plot_column].tricontour(mc_sheet['LONGITUDE_x'].astype(float),mc_sheet['Potdens'].astype(float),mc_sheet['Potdens'].astype(float), linewidths=0.5, triangles=triang.triangles, mask=triang.mask,levels=[27, 27.72], colors = "black")
  ax[plot_row,plot_column].plot(mc_sheet['LONGITUDE_x'].astype(float), mc_sheet['Potdens'].astype(float), 'ko', markersize=0.5)
  ax[plot_row,plot_column].plot(odz_n2o['LONGITUDE_x'].astype(float), odz_n2o['Potdens'].astype(float), 'wx', markersize=2)
  ax[plot_row,plot_column].clabel(dens_cntr, inline=True, fontsize=6, manual=[(-124, 27), (-124, 27.72)])

  ax[plot_row,plot_column].set(xlim=(-153, -77), ylim=(26, 28))
  ax[plot_row,plot_column].invert_yaxis()
  ax[plot_row,plot_column].set_xlabel("Longitude (degrees)", fontsize=10)
  ax[plot_row, plot_column].set_ylabel(r"$\sigma_{\Theta}$", fontsize=10)
  ax[plot_row,plot_column].set_title(plot_name, fontsize = 10)
  return cntr2

fig, ax = plt.subplots(4, 3) # Make twelve plots
fig.set_size_inches(13.5, 7.5) # Size of the panel
fig.subplots_adjust(hspace = 0.8, wspace = 0.5) # Space between plots

mc_plot("$\mathrm{N_2O_{background}}$ - Mean Stdev", "Preformed Ratio - Mean Stdev", 0, 0)
mc_plot("$\mathrm{N_2O_{background}}$", "Preformed Ratio", 0, 1)
mc_plot("$\mathrm{N_2O_{background}}$ + Mean Stdev", "Preformed Ratio + Mean Stdev", 0, 2)

mc_plot("$\mathrm{N_2O_{AOA}}$ - Mean Stdev", "AOA Ratio - Mean Stdev", 1, 0)
mc_plot("$\mathrm{N_2O_{AOA}}$", "AOA Ratio", 1, 1)
mc_plot("$\mathrm{N_2O_{AOA}}$ + Mean Stdev", "AOA Ratio + Mean Stdev", 1, 2)

mc_plot("$\mathrm{N_2O_{ID}}$ - Mean Stdev", "ID Ratio - Mean Stdev", 2, 0)
mc_plot("$\mathrm{N_2O_{ID}}$", "ID Ratio", 2, 1)
mc_plot("$\mathrm{N_2O_{ID}}$ + Mean Stdev", "ID Ratio + Mean Stdev", 2, 2)

mc_plot("$\mathrm{N_2O_{ODZ}}$ - Mean Stdev", "CD Ratio - Mean Stdev", 3, 0)
mc_plot("$\mathrm{N_2O_{ODZ}}$", "ODZ Ratio", 3, 1)
cntr2_4 = mc_plot("$\mathrm{N_2O_{ODZ}}$ + Mean Stdev", "CD Ratio + Mean Stdev", 3, 2)

ax[0, 0].text(-0.2, 1.21, 'A1.', transform=ax[0, 0].transAxes, fontsize=10, va='top', ha='center')
ax[0, 1].text(-0.2, 1.21, 'B1.', transform=ax[0, 1].transAxes, fontsize=10, va='top', ha='center')
ax[0, 2].text(-0.2, 1.21, 'C1.', transform=ax[0, 2].transAxes, fontsize=10, va='top', ha='center')
ax[1, 0].text(-0.2, 1.21, 'A2.', transform=ax[1, 0].transAxes, fontsize=10, va='top', ha='center')
ax[1, 1].text(-0.2, 1.21, 'B2.', transform=ax[1, 1].transAxes, fontsize=10, va='top', ha='center')
ax[1, 2].text(-0.2, 1.21, 'C2.', transform=ax[1, 2].transAxes, fontsize=10, va='top', ha='center')
ax[2, 0].text(-0.2, 1.21, 'A3.', transform=ax[2, 0].transAxes, fontsize=10, va='top', ha='center')
ax[2, 1].text(-0.2, 1.21, 'B3.', transform=ax[2, 1].transAxes, fontsize=10, va='top', ha='center')
ax[2, 2].text(-0.2, 1.21, 'C3.', transform=ax[2, 2].transAxes, fontsize=10, va='top', ha='center')
ax[3, 0].text(-0.2, 1.21, 'A4.', transform=ax[3, 0].transAxes, fontsize=10, va='top', ha='center')
ax[3, 1].text(-0.2, 1.21, 'B4.', transform=ax[3, 1].transAxes, fontsize=10, va='top', ha='center')
ax[3, 2].text(-0.2, 1.21, 'C4.', transform=ax[3, 2].transAxes, fontsize=10, va='top', ha='center')

cbar_ax = fig.add_axes([0.93, 0.13, 0.01, 0.74])
ticks = np.arange(-0.35, 1.25 + 0.2, 0.2)
colorbar = fig.colorbar(cntr2_4, cax=cbar_ax, ticks=ticks)
colorbar.set_label(label="Fraction", fontsize=10, rotation=270, labelpad=15)

plt.savefig('04182025_3x4MonteCarlo.jpeg', dpi=2000)
files.download('04182025_3x4MonteCarlo.jpeg')

plt.show()

In [ ]:
# 4/12/2025 Difference between results when N2O_AOA-d18O = 34 versus N2O_AOA-d18O = 46
# If not re-running model and plotting N2O fractions (just pulls data from the compiled Google Sheet specified in next line)
post_merge = import_sheet_as_df(final_gp16_regression_model, "9 Final Product Colab")

# Four ODV Plots with potential density as the y-axis
# Import necessary packages and functions
from matplotlib import tri
from sklearn.preprocessing import MinMaxScaler
from google.colab import files

# Normalize longitude and depth data before triangulation
scaler = MinMaxScaler()
scaled_long = scaler.fit_transform(post_merge['LONGITUDE_x'].astype(float).values.reshape(-1,1))
scaled_depth = scaler.fit_transform(post_merge['Potdens'].astype(float).values.reshape(-1,1))

# Create triangles that are later called in the plot line
triang=tri.Triangulation(scaled_long.flatten(), scaled_depth.flatten())
triang_analyze=tri.TriAnalyzer(triang)
mask=triang_analyze.get_flat_tri_mask()
triang.set_mask(mask)

def ratio_plot(plot_name, variable, plot_row, plot_column):
  '''
  Function that takes in the plot title, variable being plotted,
  and the position (row/column) of the specific subplot
  '''
  # Plot data
  cntr2 = ax[plot_row,plot_column].tricontourf(post_merge['LONGITUDE_x'].astype(float), post_merge['Potdens'].astype(float), post_merge[variable].astype(float), triangles=triang.triangles, mask=triang.mask,levels=np.linspace(-0.12,0.12,51,endpoint=True), extend="neither", cmap="rainbow", vmin=-0.12, vmax=0.12, fontsize=15)
  ax[plot_row,plot_column].plot(post_merge['LONGITUDE_x'].astype(float), post_merge['Potdens'].astype(float), 'ko', markersize=0.5)
  ax[plot_row,plot_column].plot(odz_n2o['LONGITUDE_x'].astype(float), odz_n2o['Potdens'].astype(float), 'wx', markersize=2)

  # Plot density contours
  dens_cntr = ax[plot_row,plot_column].tricontour(post_merge['LONGITUDE_x'].astype(float),post_merge['Potdens'].astype(float),post_merge['Potdens'].astype(float), linewidths=1.0, triangles=triang.triangles, mask=triang.mask,levels=[27, 27.72], colors = "black")
  ax[plot_row,plot_column].clabel(dens_cntr, inline=True, fontsize=6, manual=[(-123.5, 27), (-123.5, 27.72)])

  ax[plot_row,plot_column].set(xlim=(-152.9, -77.1), ylim=(26, 27.9))
  ax[plot_row,plot_column].invert_yaxis()
  ax[plot_row,plot_column].set_xlabel("Longitude (degrees)", fontsize=11)
  ax[plot_row, plot_column].set_ylabel(r"$\sigma_{\Theta}$", fontsize=11)
  ax[plot_row,plot_column].set_title(plot_name, fontsize = 11)
  ax[plot_row, plot_column].tick_params(axis='both', which='major', labelsize=11)
  return cntr2

# Make four plots
fig, ax = plt.subplots(2, 2)
fig.set_size_inches(14, 5) # Size of the panel
fig.subplots_adjust(hspace = 0.6, wspace = 0.25) # Space between plots

# Call the function four times for each plot
ratio_plot(r"$\mathrm{N_2O_{background}}$", "Diff Preformed (46 - 34)", 0, 0)
ratio_plot(r"$\mathrm{N_2O_{AOA}}$", "Diff AOA (46 - 34)", 0, 1)
ratio_plot(r"$\mathrm{N_2O_{ID}}$", "Diff ID Ratio (46 - 34)", 1, 0)
cntr2_4 = ratio_plot(r"$\mathrm{N_2O_{ODZ}}$", "Diff ODZ Ratio (46 - 34)", 1, 1)

# Place 'A.' above the top left subplot
ax[0, 0].text(0, 1.03, 'A.', transform=ax[0, 0].transAxes, fontsize=11, va='bottom', ha='left')
ax[0, 1].text(0, 1.03, 'B.', transform=ax[0, 1].transAxes, fontsize=11, va='bottom', ha='left')
ax[1, 0].text(0, 1.03, 'C.', transform=ax[1, 0].transAxes, fontsize=11, va='bottom', ha='left')
ax[1, 1].text(0, 1.03, 'D.', transform=ax[1, 1].transAxes, fontsize=11, va='bottom', ha='left')

# Colorbar
cbar_ax = fig.add_axes([0.93, 0.13, 0.01, 0.74])
colorbar = fig.colorbar(cntr2_4, cax=cbar_ax, ticks=np.linspace(-0.12, 0.12, 9))
colorbar.set_label(label="Fractional Difference", fontsize=11, rotation=270, labelpad=15)

plt.savefig('04182025_46-34diff.jpeg', dpi=2000)
files.download('04182025_46-34diff.jpeg')

plt.show()


In [ ]:
# Original 2 x 2 plot with the four end members and their respective fractions
# If not re-running model and plotting N2O fractions (just pulls data from the compiled Google Sheet specified in next line)
post_merge = import_sheet_as_df(final_gp16_regression_model, "10 Final Product Colab")

# Four ODV Plots with potential density as the y-axis
# Import necessary packages and functions
from matplotlib import tri
from sklearn.preprocessing import MinMaxScaler
from google.colab import files

# Normalize longitude and depth data before triangulation
scaler = MinMaxScaler()
scaled_long = scaler.fit_transform(post_merge['LONGITUDE_x'].astype(float).values.reshape(-1,1))
scaled_depth = scaler.fit_transform(post_merge['Potdens'].astype(float).values.reshape(-1,1))

# Create triangles that are later called in the plot line
triang=tri.Triangulation(scaled_long.flatten(), scaled_depth.flatten())
triang_analyze=tri.TriAnalyzer(triang)
mask=triang_analyze.get_flat_tri_mask()
triang.set_mask(mask)

def ratio_plot(plot_name, variable, plot_row, plot_column):
  '''
  Function that takes in the plot title, variable being plotted,
  and the position (row/column) of the specific subplot
  '''
  # Plot data
  cntr2 = ax[plot_row,plot_column].tricontourf(post_merge['LONGITUDE_x'].astype(float), post_merge['Potdens'].astype(float), post_merge[variable].astype(float), triangles=triang.triangles, mask=triang.mask,levels=np.linspace(0,1.0000001,51,endpoint=True), extend="neither", cmap="rainbow", vmin=0, vmax=1.0000001, fontsize=15)
  ax[plot_row,plot_column].plot(post_merge['LONGITUDE_x'].astype(float), post_merge['Potdens'].astype(float), 'ko', markersize=0.5)
  ax[plot_row,plot_column].plot(odz_n2o['LONGITUDE_x'].astype(float), odz_n2o['Potdens'].astype(float), 'wx', markersize=2)

  # Plot density contours
  dens_cntr = ax[plot_row,plot_column].tricontour(post_merge['LONGITUDE_x'].astype(float),post_merge['Potdens'].astype(float),post_merge['Potdens'].astype(float), linewidths=1.0, triangles=triang.triangles, mask=triang.mask,levels=[27, 27.72], colors = "black")
  ax[plot_row,plot_column].clabel(dens_cntr, inline=True, fontsize=6, manual=[(-123.5, 27), (-123.5, 27.72)])

  ax[plot_row,plot_column].set(xlim=(-152.9, -77.1), ylim=(26, 27.9))
  ax[plot_row,plot_column].invert_yaxis()
  ax[plot_row,plot_column].set_xlabel("Longitude (degrees)", fontsize=11)
  ax[plot_row, plot_column].set_ylabel(r"$\sigma_{\Theta}$", fontsize=11)
  ax[plot_row,plot_column].set_title(plot_name, fontsize = 11)
  ax[plot_row, plot_column].tick_params(axis='both', which='major', labelsize=11)
  return cntr2

# Make four plots
fig, ax = plt.subplots(2, 2)
fig.set_size_inches(12.5, 4.5) # Size of the panel
fig.subplots_adjust(hspace = 0.6, wspace = 0.3) # Space between plots

# Call the function four times for each plot
ratio_plot(r"$\mathrm{N_2O_{background}}$", "Preformed Ratio", 0, 0)
ratio_plot(r"$\mathrm{N_2O_{AOA}}$", "AOA Ratio", 0, 1)
ratio_plot(r"$\mathrm{N_2O_{ID}}$", "ID Ratio", 1, 0)
cntr2_4 = ratio_plot(r"$\mathrm{N_2O_{ODZ}}$", "ODZ Ratio", 1, 1)

# Place 'A.' above the top left subplot
ax[0, 0].text(0, 1.03, 'A.', transform=ax[0, 0].transAxes, fontsize=11, va='bottom', ha='left')
ax[0, 1].text(0, 1.03, 'B.', transform=ax[0, 1].transAxes, fontsize=11, va='bottom', ha='left')
ax[1, 0].text(0, 1.03, 'C.', transform=ax[1, 0].transAxes, fontsize=11, va='bottom', ha='left')
ax[1, 1].text(0, 1.03, 'D.', transform=ax[1, 1].transAxes, fontsize=11, va='bottom', ha='left')

# Colorbar
cbar_ax = fig.add_axes([0.93, 0.13, 0.01, 0.74])
colorbar = fig.colorbar(cntr2_4, cax=cbar_ax, ticks=np.linspace(0, 1, 6))
colorbar.set_label(label="Fraction", fontsize=11, rotation=270, labelpad=20)

plt.savefig('04242025_2x2basecase(AOA-d18O = 34).jpeg', dpi=2000)
files.download('04242025_2x2basecase(AOA-d18O = 34).jpeg')

plt.show()

In [ ]:
# 2x2 Plot of four end members and their N2O concentrations instead of fractions
# Code uses data from Google Sheet specified in the next line
post_merge = import_sheet_as_df(final_gp16_regression_model, "10 Final Product Colab")

# Four ODV Plots with potential density as the y-axis
# Import necessary packages and functions
from matplotlib import tri
from sklearn.preprocessing import MinMaxScaler
from google.colab import files

# Normalize longitude and depth data before triangulation
scaler = MinMaxScaler()
scaled_long = scaler.fit_transform(post_merge['LONGITUDE_x'].astype(float).values.reshape(-1,1))
scaled_depth = scaler.fit_transform(post_merge['Potdens'].astype(float).values.reshape(-1,1))

# Create triangles that are later called in the plot line
triang=tri.Triangulation(scaled_long.flatten(), scaled_depth.flatten())
triang_analyze=tri.TriAnalyzer(triang)
mask=triang_analyze.get_flat_tri_mask()
triang.set_mask(mask)

def ratio_plot(plot_name, variable, plot_row, plot_column):
  '''
  Function that takes in the plot title, variable being plotted,
  and the position (row/column) of the specific subplot
  '''
  # Plot data
  cntr2 = ax[plot_row,plot_column].tricontourf(post_merge['LONGITUDE_x'].astype(float), post_merge['Potdens'].astype(float), post_merge[variable].astype(float), triangles=triang.triangles, mask=triang.mask,levels=np.linspace(0,80,51,endpoint=True), extend="neither", cmap="rainbow", vmin=0, vmax=80, fontsize=15)
  ax[plot_row,plot_column].plot(post_merge['LONGITUDE_x'].astype(float), post_merge['Potdens'].astype(float), 'ko', markersize=0.5)
  ax[plot_row,plot_column].plot(odz_n2o['LONGITUDE_x'].astype(float), odz_n2o['Potdens'].astype(float), 'wx', markersize=2)

  # Plot density contours
  dens_cntr = ax[plot_row,plot_column].tricontour(post_merge['LONGITUDE_x'].astype(float),post_merge['Potdens'].astype(float),post_merge['Potdens'].astype(float), linewidths=1.0, triangles=triang.triangles, mask=triang.mask,levels=[27, 27.72], colors = "black")
  ax[plot_row,plot_column].clabel(dens_cntr, inline=True, fontsize=6, manual=[(-123.5, 27), (-123.5, 27.72)])

  ax[plot_row,plot_column].set(xlim=(-152.9, -77.1), ylim=(26, 27.9))
  # ax[plot_row,plot_column].set(xlim=(np.min(post_merge['LONGITUDE_x'].astype(float)) - 0.5, np.max(post_merge['LONGITUDE_x'].astype(float))+ 0.5), ylim=(26, 27.9))
  ax[plot_row,plot_column].invert_yaxis()
  ax[plot_row,plot_column].set_xlabel("Longitude (degrees)", fontsize=11)
  ax[plot_row, plot_column].set_ylabel(r"$\sigma_{\Theta}$", fontsize=11)
  ax[plot_row,plot_column].set_title(plot_name, fontsize = 11)
  ax[plot_row, plot_column].tick_params(axis='both', which='major', labelsize=11)
  return cntr2

# Make four plots
fig, ax = plt.subplots(2, 2)
fig.set_size_inches(12.5, 4.5) # Size of the panel
fig.subplots_adjust(hspace = 0.6, wspace = 0.3) # Space between plots

# Call the function four times for each plot
ratio_plot(r"$\mathrm{[N_2O]_{background}}$", "Preformed Conc", 0, 0)
ratio_plot(r"$\mathrm{[N_2O]_{AOA}}$", "AOA Conc", 0, 1)
ratio_plot(r"$\mathrm{[N_2O]_{ID}}$", "ID Conc", 1, 0)
cntr2_4 = ratio_plot(r"$\mathrm{[N_2O]_{ODZ}}$", "ODZ Conc", 1, 1)

# Place 'A.' above the top left subplot
ax[0, 0].text(0, 1.03, 'A.', transform=ax[0, 0].transAxes, fontsize=11, va='bottom', ha='left')
ax[0, 1].text(0, 1.03, 'B.', transform=ax[0, 1].transAxes, fontsize=11, va='bottom', ha='left')
ax[1, 0].text(0, 1.03, 'C.', transform=ax[1, 0].transAxes, fontsize=11, va='bottom', ha='left')
ax[1, 1].text(0, 1.03, 'D.', transform=ax[1, 1].transAxes, fontsize=11, va='bottom', ha='left')

# Colorbar
cbar_ax = fig.add_axes([0.93, 0.13, 0.01, 0.74])
colorbar = fig.colorbar(cntr2_4, cax=cbar_ax, ticks=np.linspace(0, 80, 9))
colorbar.set_label(label=r"$\mathrm{[N_2O]}$", fontsize=11, rotation=270, labelpad=20)

plt.savefig('04162025_meanstate.jpeg', dpi=2000)
files.download('04162025_meanstate.jpeg')

plt.show()

In [ ]:
# 4x2 Monte Carlo plots where left column is Monte Carlo mean state
# right column is the base case - monte carlo mean state
# Uses data from Google Sheet specified in next line
mc_sheet = import_sheet_as_df(final_gp16_regression_model, "10 Final Product Colab")
odz_n2o = mc_sheet[(mc_sheet['OXYGEN'].astype(float) < 5) & (mc_sheet['Potdens'].astype(float) > 26.2) & (mc_sheet['Potdens'].astype(float) < 26.8)]

from matplotlib import tri
from sklearn.preprocessing import MinMaxScaler
from google.colab import files

# NORMALIZE DATA BEFORE TRIANGULATION
scaler = MinMaxScaler()
scaled_long = scaler.fit_transform(mc_sheet['LONGITUDE_x'].astype(float).values.reshape(-1,1))
scaled_depth = scaler.fit_transform(mc_sheet['Potdens'].astype(float).values.reshape(-1,1))

triang=tri.Triangulation(scaled_long.flatten(), scaled_depth.flatten())
triang_analyze=tri.TriAnalyzer(triang)
mask=triang_analyze.get_flat_tri_mask()
triang.set_mask(mask)

def mc_plot(plot_name, variable, plot_row, plot_column):
  cntr2 = ax[plot_row,plot_column].tricontourf(mc_sheet['LONGITUDE_x'].astype(float),mc_sheet['Potdens'].astype(float),mc_sheet[variable].astype(float), triangles=triang.triangles, mask=triang.mask,levels=np.linspace(-0.35,1.35,51,endpoint=True), extend="neither", cmap="rainbow", vmin=-0.35, vmax=1.35, fontsize=15)
  # colorbar = fig.colorbar(cntr2, ax=ax[plot_row,plot_column])

  dens_cntr = ax[plot_row,plot_column].tricontour(mc_sheet['LONGITUDE_x'].astype(float),mc_sheet['Potdens'].astype(float),mc_sheet['Potdens'].astype(float), linewidths=0.5, triangles=triang.triangles, mask=triang.mask,levels=[27, 27.72], colors = "black")
  ax[plot_row,plot_column].plot(mc_sheet['LONGITUDE_x'].astype(float), mc_sheet['Potdens'].astype(float), 'ko', markersize=0.5)
  ax[plot_row,plot_column].plot(odz_n2o['LONGITUDE_x'].astype(float), odz_n2o['Potdens'].astype(float), 'wx', markersize=2)
  ax[plot_row,plot_column].clabel(dens_cntr, inline=True, fontsize=6, manual=[(-124, 27), (-124, 27.72)])

  ax[plot_row,plot_column].set(xlim=(-153, -77), ylim=(26, 28))
  ax[plot_row,plot_column].invert_yaxis()
  ax[plot_row,plot_column].set_xlabel("Longitude (degrees)", fontsize=10)
  ax[plot_row, plot_column].set_ylabel(r"$\sigma_{\Theta}$", fontsize=10)
  ax[plot_row,plot_column].set_title(plot_name, fontsize = 10)
  return cntr2

fig, ax = plt.subplots(4, 2) # Make twelve plots
fig.set_size_inches(13, 10) # Size of the panel
fig.subplots_adjust(hspace = 0.8, wspace = 0.5) # Space between plots

mc_plot(r"Monte Carlo Mean State $\mathrm{N_2O_{background}}$", "MC Mean Preformed Ratio", 0, 0)
mc_plot(r"$\mathrm{N_2O_{background}}$ - Monte Carlo Mean State $\mathrm{N_2O_{background}}$", "Pre Base Case - Mean State", 0, 1)

mc_plot(r"Monte Carlo Mean State $\mathrm{N_2O_{AOA}}$", "MC Mean AOA Ratio", 1, 0)
mc_plot(r"$\mathrm{N_2O_{AOA}}$ - Monte Carlo Mean State $\mathrm{N_2O_{AOA}}$", "AOA Base Case - Mean State", 1, 1)

mc_plot(r"Monte Carlo Mean State $\mathrm{N_2O_{ID}}$", "MC Mean ID Ratio", 2, 0)
mc_plot(r"$\mathrm{N_2O_{ID}}$ - Monte Carlo Mean State $\mathrm{N_2O_{ID}}$", "ID Base Case - Mean State", 2, 1)

mc_plot(r"Monte Carlo Mean State $\mathrm{N_2O_{ODZ}}$", "MC Mean CD Ratio", 3, 0)
cntr2_4 = mc_plot(r"$\mathrm{N_2O_{ODZ}}$ - Monte Carlo Mean State $\mathrm{N_2O_{ODZ}}$", "CD Base Case - Mean State", 3, 1)



ax[0, 0].text(-0.2, 1.21, 'A1.', transform=ax[0, 0].transAxes, fontsize=10, va='top', ha='center')
ax[0, 1].text(-0.2, 1.21, 'B1.', transform=ax[0, 1].transAxes, fontsize=10, va='top', ha='center')
ax[1, 0].text(-0.2, 1.21, 'A2.', transform=ax[1, 0].transAxes, fontsize=10, va='top', ha='center')
ax[1, 1].text(-0.2, 1.21, 'B2.', transform=ax[1, 1].transAxes, fontsize=10, va='top', ha='center')
ax[2, 0].text(-0.2, 1.21, 'A3.', transform=ax[2, 0].transAxes, fontsize=10, va='top', ha='center')
ax[2, 1].text(-0.2, 1.21, 'B3.', transform=ax[2, 1].transAxes, fontsize=10, va='top', ha='center')
ax[3, 0].text(-0.2, 1.21, 'A4.', transform=ax[3, 0].transAxes, fontsize=10, va='top', ha='center')
ax[3, 1].text(-0.2, 1.21, 'B4.', transform=ax[3, 1].transAxes, fontsize=10, va='top', ha='center')

cbar_ax = fig.add_axes([0.93, 0.13, 0.01, 0.74])
ticks = np.arange(-0.35, 1.25 + 0.2, 0.2)
colorbar = fig.colorbar(cntr2_4, cax=cbar_ax, ticks=ticks)
colorbar.set_label(label="Fraction", fontsize=10, rotation=270, labelpad=10)

plt.savefig('04162025_4x2Supp.jpeg', dpi=2000)
files.download('04162025_4x2Supp.jpeg')

plt.show()